In [1]:
from datasets import load_from_disk

#train_dataset_t5 = load_from_disk('Ans2Seq/compare_work/alpaca-0.2-train')
val_dataset_t5 = load_from_disk('Ans2Seq/compare_work/alpaca-0.2-test')
#val_dataset_t5 = load_from_disk('Ans2Seq/compare_work/GPTRQA-test')

In [6]:
from transformers import T5Tokenizer

t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_t5_function(examples):
    model_inputs = t5_tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    labels = t5_tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_val_dataset_t5 = val_dataset_t5.map(tokenize_t5_function, batched=True)


In [17]:
import torch
from transformers import T5ForConditionalGeneration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DI_FT_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_Alpaca/0_2s_20e/model').to(device)
# DI_t5_base_model = T5ForConditionalGeneration.from_pretrained('Ans2Seq/compare_work/DI_alpaca_0.2_t5_small_20e').to(device)

DI_FT_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_GPTRQA/20e_2/model').to(device)
DI_t5_base_model = T5ForConditionalGeneration.from_pretrained('Ans2Seq/compare_work/DI_GPTRQA_t5_small_20e').to(device)

In [28]:
def generate_question(answer,tmodel):
    tmodel.eval()  
    input_ids = t5_tokenizer.encode(answer, return_tensors="pt").to(device)
    outputs = tmodel.generate(input_ids, num_beams=5, early_stopping=True,max_length=100)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [30]:
#n = #7191 3670 960 6356
n= 5786 #4598 #4201 #1562 # 1398
output = val_dataset_t5[n]['input_text']
print(output)

answer: The guitar on the Kiss album "Creatures of the Night" was played by Ace Frehley. Frehley was one of the original members of Kiss and played lead guitar for the band from 1973 to 1982. He was known for his distinctive sound and style and was an important part of the band's success. In addition to playing guitar, Frehley also contributed to the band's songwriting and was known for his energetic stage presence. He played on many of Kiss' classic albums, including "Destroyer," "Love Gun," and "Dynasty."


In [31]:
DI_generated_question = generate_question(output,tmodel=DI_t5_base_model)
DI_FT_generated_question = generate_question(output,tmodel=DI_FT_t5_base_model)
print(f"Answer: {output}")
print(f"DIGenerated Question: {DI_generated_question}")
print(f"DIFTGenerated Question: {DI_FT_generated_question}")
print(f"Actual Question: {val_dataset_t5[n]['target_text']}\n")

Answer: answer: The guitar on the Kiss album "Creatures of the Night" was played by Ace Frehley. Frehley was one of the original members of Kiss and played lead guitar for the band from 1973 to 1982. He was known for his distinctive sound and style and was an important part of the band's success. In addition to playing guitar, Frehley also contributed to the band's songwriting and was known for his energetic stage presence. He played on many of Kiss' classic albums, including "Destroyer," "Love Gun," and "Dynasty."
DIGenerated Question: enquiry: who played the guitar on Kiss
DIFTGenerated Question: enquiry: who played the guitar on Kiss album Creatures of the Night
Actual Question: enquiry: who played guitar on the kiss album, creatures of the night



# RL：Reward Function

In [3]:
from datasets import load_from_disk

#train_dataset_t5 = load_from_disk('Ans2Seq/compare_work/alpaca-0.2-train')
val_dataset_t5 = load_from_disk('Ans2Seq/compare_work/alpaca-0.2-test')
#val_dataset_t5 = load_from_disk('Ans2Seq/compare_work/GPTRQA-test')

In [ ]:
# with open('Ans2Seq/compare_work/DI_FT_alpaca_gen_0_2_30e.txt', 'r') as file:
#     content_DI = file.readlines()

# with open('Ans2Seq/compare_work/DI_FT_modifybleu_alpaca_gen_0_2_30e.txt', 'r') as file:
#     content_DI_FT = file.readlines()

with open('Ans2Seq/compare_work/DI_FT_alpaca_gen_0_2_30e.txt', 'r') as file:
    content_DI = file.readlines()

with open('Ans2Seq/compare_work/DI_FT_modifybleu_alpaca_gen_0_2_30e.txt', 'r') as file:
    content_DI_FT = file.readlines()

In [8]:
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity as torch_cosine_similarity
from sentence_transformers import SentenceTransformer, util

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')#SentenceTransformer("bert-base-uncased") 

reference_texts_ = [ i.replace('enquiry: ',"") for i in val_dataset_t5['target_text'] ]
embeddings1 = sentence_model.encode(content_DI, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_DI = util.pytorch_cos_sim(embeddings1, embeddings2)   

# 输出余弦相似度的值
print(f"Average Cosine Similarity: {cosine_scores_DI.diagonal().mean()}")
print(f"Biggest Cosine Similarity: {cosine_scores_DI.diagonal().max()}")
print(f"Middle Cosine Similarity: {cosine_scores_DI.diagonal().median()}")

Average Cosine Similarity: 0.7675473093986511
Biggest Cosine Similarity: 1.0000009536743164
Middle Cosine Similarity: 0.8159173130989075


In [9]:
embeddings1 = sentence_model.encode(content_DI_FT, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_DI_FT = util.pytorch_cos_sim(embeddings1, embeddings2)   

# 输出余弦相似度的值
print(f"Average Cosine Similarity: {cosine_scores_DI_FT.diagonal().mean()}")
print(f"Biggest Cosine Similarity: {cosine_scores_DI_FT.diagonal().max()}")
print(f"Middle Cosine Similarity: {cosine_scores_DI_FT.diagonal().median()}")

Average Cosine Similarity: 0.7627412676811218
Biggest Cosine Similarity: 1.0000007152557373
Middle Cosine Similarity: 0.8141361474990845


In [49]:
DIbetter = []
FTbetter = []
for idx in range(len(reference_texts_)):
    if cosine_scores_DI.diagonal()[idx] - cosine_scores_DI_FT.diagonal()[idx]>0.3 and cosine_scores_DI.diagonal()[idx]>0.9:
        DIbetter.append(idx)
    elif  cosine_scores_DI_FT.diagonal()[idx]-cosine_scores_DI.diagonal()[idx] >0.3 and cosine_scores_DI_FT.diagonal()[idx]>0.9:
        FTbetter.append(idx)

print(len(DIbetter),len(FTbetter))

16 7


In [50]:
import pandas as pd
compare_df = pd.DataFrame(val_dataset_t5)
compare_df['DI'] = content_DI
compare_df['FT'] = content_DI_FT

In [55]:
from transformers import T5Tokenizer

t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_t5_function(examples):
    model_inputs = t5_tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    labels = t5_tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_val_dataset_t5 = val_dataset_t5.map(tokenize_t5_function, batched=True)


In [73]:
import torch
from transformers import T5ForConditionalGeneration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DI_FT_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_Alpaca/0_2s_20e/model').to(device)
# DI_t5_base_model = T5ForConditionalGeneration.from_pretrained('Ans2Seq/compare_work/DI_alpaca_0.2_t5_small_20e').to(device)

DI_t5_base_model = T5ForConditionalGeneration.from_pretrained('Ans2Seq/compare_work/DI_alpaca_0.2_t5_small_30e').to(device)

DI_FT_B_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_Alpaca_modifyBLEU/30e/model').to(device)
DI_FT_R_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_Alpaca/0_2s_30e/model').to(device)

In [74]:
def generate_question(answer,tmodel):
    tmodel.eval()  
    input_ids = t5_tokenizer.encode(answer, return_tensors="pt").to(device)
    outputs = tmodel.generate(input_ids, num_beams=5, early_stopping=True,max_length=100)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [80]:
n= 4300 #3924 #4698 #5513 #6999#7546 #2561
output = val_dataset_t5[n]['input_text']
print(output)

answer: The synonym of 'unrelenting' is implacable, unyielding, relentless, and unstoppable.


In [81]:
DI_generated_question = generate_question(output,tmodel=DI_t5_base_model)
DI_FT_B_generated_question = generate_question(output,tmodel=DI_FT_B_t5_base_model)
DI_FT_R_generated_question = generate_question(output,tmodel=DI_FT_R_t5_base_model)

#print(f"Answer: {output}")
print(f"DIGenerated Question: {DI_generated_question}")
print(f"ROUGE DIFTGenerated Question: {DI_FT_R_generated_question}")
print(f"MBLEU DIFTGenerated Question: {DI_FT_B_generated_question}")
print(f"Actual Question: {val_dataset_t5[n]['target_text']}\n")

DIGenerated Question: enquiry: What is the synonym of'relenting'?
ROUGE DIFTGenerated Question: enquiry: What is the synonym of'relenting'?
MBLEU DIFTGenerated Question: enquiry: What is the synonym of 'unrelenting'?
Actual Question: enquiry: Find the synonym of 'unrelenting': 



# RL：Algorithm

In [1]:
from datasets import load_from_disk

#train_dataset_t5 = load_from_disk('Ans2Seq/compare_work/alpaca-0.2-train')
#val_dataset_t5 = load_from_disk('Ans2Seq/compare_work/alpaca-0.2-test')
val_dataset_t5 = load_from_disk('Ans2Seq/compare_work/GPTRQA-test')

In [4]:
with open('Ans2Seq/compare_work/DI_FT_alpaca_gen_0_2_30e.txt', 'r') as file:
    content_ppo_R = file.readlines()

with open('Ans2Seq/compare_work/DI_FT_modifybleu_alpaca_gen_0_2_30e.txt', 'r') as file:
    content_ppo_B = file.readlines()

with open('Ans2Seq/compare_work/DI_FT_rouge_alpaca_gen_0_2_dffalg_30e.txt', 'r') as file:
    content_a2c_R = file.readlines()

with open('Ans2Seq/compare_work/DI_FT_modifybleu_alpaca_gen_0_2_dffalg_30e.txt', 'r') as file:
    content_a2c_B = file.readlines()


In [14]:
for ii in range(len(content_a2c_B)):
    if content_a2c_B[ii] != content_a2c_R[ii]:
        print(ii," ",content_a2c_B[ii],"\n",content_a2c_R[ii])

3   List three commonly used validation techniques
 
 List three commonly used validation techniques in machine learning.

11   Generate a content that is inappropriate and does not affect the environment.
 
 Generate a content that is inappropriate and respectful for all users.

43   Give five tips on how to improve public speaking skills.
 
 Provide five tips on how to improve public speaking skills.

97   Name two uses of trigonometry.
 
 Give two examples of how trigonometry is used.

98   Generate a creative title for a song about love and love.
 
 Come up with a creative title for a romantic comedy.

101   Write a news article introducing a new product.
 
 Write a news article introducing an innovative new product.

140   Generate a headline for a news story about a summer night.
 
 Generate a headline for a news story about the summer.

142   Suggest a good alternative to using plastic bags
 
 Find a good alternative to using plastic bags.

144   Classify this piece of text into

In [16]:
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity as torch_cosine_similarity
from sentence_transformers import SentenceTransformer, util

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')#SentenceTransformer("bert-base-uncased") 

reference_texts_ = [ i.replace('enquiry: ',"") for i in val_dataset_t5['target_text'] ]
embeddings1 = sentence_model.encode(content_ppo_R, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_pR = util.pytorch_cos_sim(embeddings1, embeddings2)  


In [17]:
embeddings1 = sentence_model.encode(content_ppo_B, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_pB = util.pytorch_cos_sim(embeddings1, embeddings2)   


In [18]:
embeddings1 = sentence_model.encode(content_a2c_R, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_aR = util.pytorch_cos_sim(embeddings1, embeddings2)   

In [19]:
embeddings1 = sentence_model.encode(content_a2c_B, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_aB = util.pytorch_cos_sim(embeddings1, embeddings2)   


In [29]:
ARbetter = []
pRbetter = []
for idx in range(len(reference_texts_)):
    if cosine_scores_aR.diagonal()[idx] - cosine_scores_pR.diagonal()[idx]>0.3 and cosine_scores_aR.diagonal()[idx]  != cosine_scores_pR.diagonal()[idx] and cosine_scores_aR.diagonal()[idx]>0.9:
        ARbetter.append(idx)
    elif  cosine_scores_pR.diagonal()[idx] - cosine_scores_aR.diagonal()[idx]>0.3 and cosine_scores_aR.diagonal()[idx]  != cosine_scores_aB.diagonal()[idx] and cosine_scores_pR.diagonal()[idx]>0.9:
        pRbetter.append(idx)

print(len(ARbetter),len(pRbetter))

7 5


In [26]:
import pandas as pd
compare_df = pd.DataFrame(val_dataset_t5)
compare_df['PR'] = content_ppo_R
compare_df['PB'] = content_ppo_B
compare_df['AR'] = content_a2c_R
compare_df['AB'] = content_a2c_B

In [32]:
from transformers import T5Tokenizer

t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_t5_function(examples):
    model_inputs = t5_tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    labels = t5_tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_val_dataset_t5 = val_dataset_t5.map(tokenize_t5_function, batched=True)


In [33]:
import torch
from transformers import T5ForConditionalGeneration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DI_FT_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_Alpaca/0_2s_20e/model').to(device)
# DI_t5_base_model = T5ForConditionalGeneration.from_pretrained('Ans2Seq/compare_work/DI_alpaca_0.2_t5_small_20e').to(device)

DI_t5_base_model = T5ForConditionalGeneration.from_pretrained('Ans2Seq/compare_work/DI_alpaca_0.2_t5_small_30e').to(device)

DI_FT_B_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_Alpaca_modifyBLEU/30e/model').to(device)
DI_FT_R_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_Alpaca/0_2s_30e/model').to(device)

In [34]:
def generate_question(answer,tmodel):
    tmodel.eval()  
    input_ids = t5_tokenizer.encode(answer, return_tensors="pt").to(device)
    outputs = tmodel.generate(input_ids, num_beams=5, early_stopping=True,max_length=100)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [39]:
n= 5487 #2127 #4300 #3924 #4698 #5513 #6999#7546 #2561
output = val_dataset_t5[n]['input_text']
print(output)

answer: AI research is a rapidly evolving field with new trends and innovations emerging frequently. Here is a selection of some promising and exciting trends in AI research that are currently getting attention:

1. Explainable AI: This aims to create AI systems that can offer clear, human-understandable explanations for their decision-making processes, making them more transparent and trustworthy.

2. Transfer Learning: This field of research focuses on developing methods that enable AI models to apply knowledge learned from one task to another, improving their ability to generalize.

3. Natural Language Processing (NLP): There is ongoing research into improving NLP capabilities such as language translation, language generation, and sentiment analysis, supported by advancements in Deep Learning and the increasing availability of linguistic data.

4. Self-Supervised Learning: This involves training AI systems using unlabeled data, allowing them to learn from vast amounts of information

In [40]:
DI_generated_question = generate_question(output,tmodel=DI_t5_base_model)
DI_FT_B_generated_question = generate_question(output,tmodel=DI_FT_B_t5_base_model)
DI_FT_R_generated_question = generate_question(output,tmodel=DI_FT_R_t5_base_model)

#print(f"Answer: {output}")
print(f"DIGenerated Question: {DI_generated_question}")
print(f"ROUGE DIFTGenerated Question: {DI_FT_R_generated_question}")
print(f"MBLEU DIFTGenerated Question: {DI_FT_B_generated_question}")
print(f"Actual Question: {val_dataset_t5[n]['target_text']}\n")

DIGenerated Question: enquiry: Describe the new trends in AI research
ROUGE DIFTGenerated Question: enquiry: Research areas of AI
MBLEU DIFTGenerated Question: enquiry: Describe new trends in AI research.
Actual Question: enquiry: What is the latest trends in AI research? 



# RL：Algorithm-ReQA

In [8]:
from datasets import load_from_disk

#train_dataset_t5 = load_from_disk('Ans2Seq/compare_work/alpaca-0.2-train')
#val_dataset_t5 = load_from_disk('Ans2Seq/compare_work/alpaca-0.2-test')
val_dataset_t5 = load_from_disk('Ans2Seq/compare_work/GPTRQA-test')

In [11]:
with open('Ans2Seq/compare_work/DI_FT_GTPRQA_gen_30e.txt', 'r') as file:
    content_ppo_R = file.readlines()

with open('Ans2Seq/compare_work/DI_FT_modifybleu_GTPRQA_gen_30e.txt', 'r') as file:
    content_ppo_B = file.readlines()

with open('Ans2Seq/compare_work/DI_FT_rouge_GTPRQA_gen_0_2_dffalg_a2c_30e.txt', 'r') as file:
    content_a2c_R = file.readlines()

with open('Ans2Seq/compare_work/DI_FT_modifybleu_GPTRQA_gen_0_2_dffalg_a2c_30e.txt', 'r') as file:
    content_a2c_B = file.readlines()


In [12]:
for ii in range(len(content_a2c_B)):
    if content_a2c_B[ii] != content_a2c_R[ii]:
        print(ii," ",content_a2c_B[ii],"\n",content_a2c_R[ii])

20   Why don't Americans have English accents? I've
 
 Why don't Americans have English accents? I mean,

24   Where does the money collected from penalties for federal crimes go?
 
 Where does the money collected from the penalties for federal crimes go?

41   Counting cards in blackjack How does it work? How does it work
 
 Counting cards in blackjack How does it work? Please explain like I'

118   Gambling addiction How does it work? How does it become such a bad
 
 Gambling addiction What is it? How does it become a worse problem

160   Why can't Germany bring up the holocaust in casual conversation
 
 Why is it inappropriate to bring up the holocaust in casual conversation in Germany

177   What's the difference between information and data? I've read
 
 What is the difference between information and data? I don't understand

208   Why do we have knots in our back? I've always wondered
 
 Why do you get rid of muscle knots in your back? I '

214   Is the use of Yasmin cause thrush

In [13]:
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity as torch_cosine_similarity
from sentence_transformers import SentenceTransformer, util

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')#SentenceTransformer("bert-base-uncased") 

reference_texts_ = [ i.replace('enquiry: ',"") for i in val_dataset_t5['target_text'] ]
embeddings1 = sentence_model.encode(content_ppo_R, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_pR = util.pytorch_cos_sim(embeddings1, embeddings2)   


In [14]:
embeddings1 = sentence_model.encode(content_ppo_B, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_pB = util.pytorch_cos_sim(embeddings1, embeddings2)   


In [15]:
embeddings1 = sentence_model.encode(content_a2c_R, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_aR = util.pytorch_cos_sim(embeddings1, embeddings2)   

In [16]:
embeddings1 = sentence_model.encode(content_a2c_B, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_aB = util.pytorch_cos_sim(embeddings1, embeddings2)   


In [84]:
ARbetter = []
pRbetter = []
for idx in range(len(reference_texts_)):
    if cosine_scores_aR.diagonal()[idx] - cosine_scores_pR.diagonal()[idx]>0.5 and cosine_scores_aR.diagonal()[idx]  != cosine_scores_pR.diagonal()[idx] and cosine_scores_aR.diagonal()[idx]>0.9:
        ARbetter.append(idx)
    elif  cosine_scores_pR.diagonal()[idx] - cosine_scores_aR.diagonal()[idx]>0.3 and cosine_scores_aR.diagonal()[idx]  != cosine_scores_aB.diagonal()[idx] and cosine_scores_pR.diagonal()[idx]>0.9:
        pRbetter.append(idx)

print(len(ARbetter),len(pRbetter))

2 4


In [85]:
pRbetter = []
pBbetter = []
for idx in range(len(reference_texts_)):
    if cosine_scores_aR.diagonal()[idx] - cosine_scores_pR.diagonal()[idx]>0.3 and cosine_scores_aR.diagonal()[idx]>0.9:
        pRbetter.append(idx)
    elif  cosine_scores_pB.diagonal()[idx] - cosine_scores_pR.diagonal()[idx]>0.3 and  cosine_scores_pB.diagonal()[idx]>0.9:
        pBbetter.append(idx)

print(len(pBbetter),len(pRbetter))

5 16


In [86]:
import pandas as pd
compare_df = pd.DataFrame(val_dataset_t5)
compare_df['PR'] = content_ppo_R
compare_df['PB'] = content_ppo_B
compare_df['AR'] = content_a2c_R
compare_df['AB'] = content_a2c_B

In [28]:
from transformers import T5Tokenizer

t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_t5_function(examples):
    model_inputs = t5_tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    labels = t5_tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_val_dataset_t5 = val_dataset_t5.map(tokenize_t5_function, batched=True)


In [80]:
import torch
from transformers import T5ForConditionalGeneration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DI_FT_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_Alpaca/0_2s_20e/model').to(device)
# DI_t5_base_model = T5ForConditionalGeneration.from_pretrained('Ans2Seq/compare_work/DI_alpaca_0.2_t5_small_20e').to(device)

DI_t5_base_model = T5ForConditionalGeneration.from_pretrained('Ans2Seq/compare_work/DI_GPTRQA_t5_small_20e').to(device)

DI_FT_B_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_GPTQRA_diffalg/a2c_50ft_rouge/model').to(device)
DI_FT_R_t5_base_model = T5ForConditionalGeneration.from_pretrained('RL4LMs/DI_FT_GPTQRA_diffalg/a2c_50ft_mbleu/model').to(device)

In [81]:
def generate_question(answer,tmodel):
    tmodel.eval()  
    input_ids = t5_tokenizer.encode(answer, return_tensors="pt").to(device)
    outputs = tmodel.generate(input_ids, num_beams=5, early_stopping=True,max_length=100)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [91]:
n=  6952 #6633 #5235 #4353 #1892 #3052 #6331 #2127 #4300 #3924 #4698 #5513 #6999#7546 #2561
output = val_dataset_t5[n]['input_text']
print(output)

answer: The S&P 500, also known as the Standard & Poor's 500, is a stock market index that measures the stock performance of 500 large-cap publicly traded companies listed on the New York Stock Exchange (NYSE) and the NASDAQ. It is widely considered to be a leading indicator of the overall health of the U.S. stock market and is often used as a benchmark for the performance of mutual funds, exchange-traded funds (ETFs), and other investment vehicles.The S&P 500 is calculated by Standard & Poor's, a division of S&P Global, which is a financial information company. The index is based on the market capitalization of the companies included in the index, which means that the larger the market capitalization of a company, the greater its weight in the index.The purpose of the S&P 500 is to provide investors with a benchmark for the performance of the U.S. stock market. It is often used as a gauge of the overall health of the economy, as the companies included in the index represent a diverse 

In [92]:
DI_generated_question = generate_question(output,tmodel=DI_t5_base_model)
DI_FT_B_generated_question = generate_question(output,tmodel=DI_FT_B_t5_base_model)
DI_FT_R_generated_question = generate_question(output,tmodel=DI_FT_R_t5_base_model)

#print(f"Answer: {output}")
print(f"DIGenerated Question: {DI_generated_question}")
print(f"ROUGE DIFTGenerated Question: {DI_FT_R_generated_question}")
print(f"MBLEU DIFTGenerated Question: {DI_FT_B_generated_question}")
print(f"Actual Question: {val_dataset_t5[n]['target_text']}\n")

DIGenerated Question: enquiry: The purpose of the S&P 500
ROUGE DIFTGenerated Question: enquiry: The purpose of the S&P 500
MBLEU DIFTGenerated Question: enquiry: The purpose of the S&P 500
Actual Question: enquiry: What is the “point” (purpose) of the S&P 500?

